# 여기서부터 prompt템플릿 추가



---



In [53]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, RetrievalQA
from langchain_upstage import ChatUpstage
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.question_answering import load_qa_chain

In [54]:
template_classifier_prompt = PromptTemplate(
    input_variables=["question"],
    template="""
다음 질문을 보고, 어떤 템플릿을 사용하는 것이 가장 적절한지 분류하세요.
선택지는 다음과 같습니다:

- summary: 채용공고 내용을 요약하는 질문
- recommendation: 조건에 맞는 공고를 추천해달라는 질문
- comparison: 공고들을 비교해달라는 질문
- clean_list: 추천을 하면서 공고 리스트를 정리해서 보여달라는 복합 요청
- general: 위에 해당하지 않는 일반 질문

답변은 위 5개 중 하나의 단어로만 출력하세요. 설명은 하지 마세요.

질문: "{question}"
답변:
"""
)
summary_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
당신은 채용 공고 요약 전문가입니다.

아래 채용 공고 내용을 읽고, 다음 정보를 요약해 주세요:
1. 회사명
2. 포지션 요약
3. 주요 업무
4. 자격 요건 (필수 및 우대)
5. 복지 및 혜택

[공고 내용]
{context}

[요약]
"""
)
recommendation_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
당신은 커리어 코치입니다.

아래 사용자의 상황에 맞는 채용 공고를 3~5개 추천해 주세요.
각 공고는 다음 항목으로 정리해 주세요:
- 회사명
- 포지션
- 추천 이유 (사용자의 질문과 연결 지어 설명 - 문단 구분하기)

[사용자 질문]
{question}

[공고 목록]
{context}

[추천 결과]
"""
)
comparison_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
당신은 취업 컨설턴트입니다.

아래의 두 개 이상의 채용 공고를 다음 기준에 따라 비교해 주세요:
- 회사 및 포지션 개요
- 주요 업무
- 자격요건 (필수/우대)
- 복지/문화
- 사용자 조건과의 적합도

[사용자 질문]
{question}

[비교 대상 공고]
{context}

[비교 결과]
"""
)

clean_list_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
당신은 커리어 코치입니다.

다음 사용자의 상황에 맞는 채용 공고를 3~5개 추천해 주세요.
각 공고는 다음 형식으로 깔끔하게 리스트 형태로 제시해 주세요.

형식 예시:
1. 회사명: OOO
   포지션: OOO
   근무지역: OOO
   마감일: OOO
   추천 이유: [사용자의 질문과 연결하여 간결하게 설명]

[사용자 질문]
{question}

[참고 가능한 채용공고 목록]
{context}

[추천 공고]
"""
)

In [55]:
# 템플릿 맵
TEMPLATES = {
    "summary": summary_prompt,
    "recommendation": recommendation_prompt,
    "comparison": comparison_prompt,
    "clean_list": clean_list_template,
}

In [56]:
# 질문 유형 분류 체인
llm = ChatUpstage(model="solar-pro")
template_classifier_chain = LLMChain(
    llm=llm,
    prompt=template_classifier_prompt,
    output_key="template"
)

def classify_question(question: str) -> str:
    result = template_classifier_chain.run(question).strip().lower()
    allowed = ["summary", "recommendation", "comparison", "clean_list", "general"]
    return result if result in allowed else "general"

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_4272\1957414946.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  template_classifier_chain = LLMChain(


In [59]:
# QA 수행
query = "7월까지 마감하는 공고 추천해줘"

query_type = classify_question(query)
prompt = TEMPLATES.get(query_type)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt} if prompt else {},
    return_source_documents=True
)

result = qa_chain.invoke({"history": [], "query": query})

In [60]:
print("[답변]")
print(result["result"])

[답변]
1. 회사명: 숨고  
   포지션: Product Team / UX Chapter  
   추천 이유: 사용자는 7월까지 마감하는 채용 공고를 찾고 있으며, 숨고의 UX Researcher 포지션은 UX 리서치, 사용자 경험 개선, 디자인 방향성 제언 등 다양한 업무를 수행할 수 있는 기회를 제공합니다. 이는 사용자가 UX/UI 디자인이나 사용자 경험 개선에 관심이 있다면 적합할 수 있습니다.

2. 회사명: 숨고  
   포지션: Product Team / PO Chapter  
   추천 이유: 이 포지션은 프로덕트 매니저로서 콘텐츠와 Referral 전략을 통해 잠재 고객을 확보하고 고객의 리텐션을 증대시키는 전략을 담당합니다. 사용자는 빠른 성장 환경에서 프로젝트 리딩 및 비즈니스 목표를 달성하는 업무에 관심이 있다면 이 포지션을 고려할 수 있습니다.

3. 회사명: Mnet Plus 사업부  
   포지션: Mnet Plus 프로덕트 기획  
   추천 이유: 이 포지션은 KPOP 팬덤과 글로벌 엔터테인먼트 산업에 관심이 있는 사용자에게 적합합니다. 데이터 기반 의사결정, 프로덕트 분석 및 기획, 다양한 이해관계자와의 협업 등을 통해 글로벌 K-Culture 소비자와 콘텐츠, 아티스트를 연결하는 디지털 접점을 구축하는 업무를 수행합니다.
